In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Basics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf

# Data
import xarray as xr
import h5py
import pickle

# Helpful
import time
import datetime
import itertools
from itertools import product
from tqdm import tqdm
import os

# My Methods
import importlib
from src.utils.CRPS import *
from src.utils.data_split import *
from src.models.EMOS import *
import data.raw.load_data_raw as ldr
import data.processed.load_data_processed as ldp
from src.models.EMOS_global.EMOS_global_load_models import *

2023-05-24 10:11:34.082061: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-05-24 10:11:34.082082: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


<IPython.core.display.Javascript object>

## Goal: Make ws10 feature

### 0. Basics

In [3]:
var_names = ["u10", "v10", "t2m", "t850", "z500"]

<IPython.core.display.Javascript object>

In [4]:
# Variables to make testing easier --> determines variable and lead_time
var = 0  # 0,...,4
lead_time = 1  # 0,...,30
forecast_date = 0
lat = 0
lon = 0

<IPython.core.display.Javascript object>

### 1. Load Dataset

In [5]:
dat_raw = ldr.load_data_raw()  # list length 5 with 2018 - 2022

# processed data
dat_train_proc_norm = ldp.load_data_all_train_proc_norm()
dat_test_proc_norm = ldp.load_data_all_test_proc_norm()

<IPython.core.display.Javascript object>

### 2. Test if mean is enough, or have to do everything from scratch?

In [6]:
u10_test_ens = (
    dat_raw[0]
    .isel(forecast_date=forecast_date, lead_time=lead_time, var=0, lat=lat, lon=lon)
    .values
)
v10_test_ens = (
    dat_raw[0]
    .isel(forecast_date=forecast_date, lead_time=lead_time, var=1, lat=lat, lon=lon)
    .values
)
u10_test_mean = (
    dat_train_proc_norm[0]
    .u10_train.isel(
        forecast_date=forecast_date, lead_time=lead_time, lat=lat, lon=lon, mean_std=0
    )
    .values
)
v10_test_mean = (
    dat_train_proc_norm[1]
    .v10_train.isel(
        forecast_date=forecast_date, lead_time=lead_time, lat=lat, lon=lon, mean_std=0
    )
    .values
)
u10_test_std = (
    dat_train_proc_norm[0]
    .u10_train.isel(
        forecast_date=forecast_date, lead_time=lead_time, lat=lat, lon=lon, mean_std=1
    )
    .values
)
v10_test_std = (
    dat_train_proc_norm[1]
    .v10_train.isel(
        forecast_date=forecast_date, lead_time=lead_time, lat=lat, lon=lon, mean_std=1
    )
    .values
)

<IPython.core.display.Javascript object>

### 3. From raw data make ws10

In [7]:
dat_raw[0].predictionsu10_X_ens = (
    dat_raw[0]
    .predictions.isel(
        forecast_date=forecast_date, lead_time=lead_time, var=0, lat=lat, lon=lon
    )
    .values
)
v10_X_ens = (
    dat_raw[0]
    .predictions.isel(
        forecast_date=forecast_date, lead_time=lead_time, var=1, lat=lat, lon=lon
    )
    .values
)

<IPython.core.display.Javascript object>

In [8]:
dat_raw[0].predictions

<xarray.DataArray 'predictions' (forecast_date: 357, ens: 50, lead_time: 32,
                                 var: 5, lat: 120, lon: 130)>
[44553600000 values with dtype=float32]
Dimensions without coordinates: forecast_date, ens, lead_time, var, lat, lon

<IPython.core.display.Javascript object>

In [14]:
dat_raw[0].predictions.isel(forecast_date=0, lead_time=0, var=0).shape

(50, 120, 130)

<IPython.core.display.Javascript object>

In [26]:
ws10_2018 = np.hypot(
    dat_raw[0].predictions.isel(lead_time=0, var=0),
    dat_raw[0].predictions.isel(lead_time=0, var=1),
)

<IPython.core.display.Javascript object>

In [18]:
ws10_2018_truth = np.hypot(
    dat_raw[0].ground_truth.isel(forecast_date=0, lead_time=0, var=0),
    dat_raw[0].ground_truth.isel(forecast_date=0, lead_time=0, var=1),
)

<IPython.core.display.Javascript object>

In [27]:
ws10_2018

<xarray.DataArray 'predictions' (forecast_date: 357, ens: 50, lat: 120, lon: 130)>
array([[[[2.464211  , 2.5051975 , 2.530376  , ..., 0.5577944 ,
          0.5903407 , 0.63572365],
         [2.4863982 , 2.4688694 , 2.4337444 , ..., 0.55735195,
          0.6233006 , 0.67398   ],
         [2.3784466 , 2.3241603 , 2.260372  , ..., 0.61965936,
          0.6557909 , 0.695246  ],
         ...,
         [1.5332146 , 1.5627002 , 1.5289203 , ..., 0.9051664 ,
          0.963259  , 1.0141015 ],
         [1.568858  , 1.4870164 , 1.2883853 , ..., 0.932049  ,
          0.97790176, 1.013755  ],
         [1.0026784 , 0.67359126, 0.53233415, ..., 0.8896815 ,
          0.908034  , 0.90010846]],

        [[2.464211  , 2.5051975 , 2.530376  , ..., 0.5577944 ,
          0.5903407 , 0.63572365],
         [2.4863982 , 2.4688694 , 2.4337444 , ..., 0.55735195,
          0.6233006 , 0.67398   ],
         [2.3784466 , 2.3241603 , 2.260372  , ..., 0.61965936,
          0.6557909 , 0.695246  ],
...
         [0.71295035, 0.4494548 , 0.5546265 , ..., 1.337214  ,
          1.3682648 , 1.4017646 ],
         [0.7905015 , 0.34083292, 0.2836684 , ..., 1.2715337 ,
          1.327162  , 1.3503352 ],
         [1.122537  , 0.7809611 , 0.35761604, ..., 1.1406059 ,
          1.2346797 , 1.2310573 ]],

        [[0.30894384, 0.28449228, 0.27381676, ..., 0.20353673,
          0.2635594 , 0.33263785],
         [0.35607412, 0.3543335 , 0.36019957, ..., 0.25120094,
          0.32963127, 0.40093192],
         [0.43685737, 0.43939131, 0.44462055, ..., 0.2868718 ,
          0.37013668, 0.44080743],
         ...,
         [0.71295035, 0.4494548 , 0.5546265 , ..., 1.337214  ,
          1.3682648 , 1.4017646 ],
         [0.7905015 , 0.34083292, 0.2836684 , ..., 1.2715337 ,
          1.327162  , 1.3503352 ],
         [1.122537  , 0.7809611 , 0.35761604, ..., 1.1406059 ,
          1.2346797 , 1.2310573 ]]]], dtype=float32)
Dimensions without coordinates: forecast_date, ens, lat, lon

<IPython.core.display.Javascript object>

In [35]:
len(ldr.load_data_raw()[0:4])

4

<IPython.core.display.Javascript object>

In [36]:
def make_ws10_train_data():
    """
    Function to process and save wind speed prediction and ground truth data.
    """

    # Initialize timer for performance tracking
    start_time = time.time()

    # Define path and file names for the h5 file to be created
    path = "/Data/Delong_BA_Data/Mean_ens_std/ws10_train.h5"
    f = h5py.File(path, "a")
    name_train = "ws10_train"
    name_truth = "ws10_truth"

    # Load raw data for the years 2018-2021
    dat_raw = ldr.load_data_raw()[0:4]

    # Compute the magnitude (absolute value) of wind speed predictions and truths
    ws10_preds = []  # list to hold prediction data
    ws10_truth = []  # list to hold ground truth data
    for year in tqdm(range(4)):
        ws10_preds.append(
            np.hypot(
                dat_raw[year].predictions.isel(var=0),
                dat_raw[year].predictions.isel(var=1),
            )
        )
        ws10_truth.append(
            np.hypot(
                dat_raw[year].ground_truth.isel(var=0),
                dat_raw[year].ground_truth.isel(var=1),
            )
        )

    # Calculate and concatenate mean and standard deviation of wind speed predictions
    ws10_preds_mean = xr.concat(
        [
            ws10_preds[0].mean(dim="ens"),
            ws10_preds[1].mean(dim="ens"),
            ws10_preds[2].mean(dim="ens"),
            ws10_preds[3].mean(dim="ens"),
        ],
        dim="forecast_date",
    )
    ws10_preds_std = xr.concat(
        [
            ws10_preds[0].std(dim="ens"),
            ws10_preds[1].std(dim="ens"),
            ws10_preds[2].std(dim="ens"),
            ws10_preds[3].std(dim="ens"),
        ],
        dim="forecast_date",
    )

    # Concatenate mean and standard deviation data along new 'mean_std' dimension
    ws_train = xr.concat([ws10_preds_mean, ws10_preds_std], dim="mean_std")
    ws_train = ws_train.transpose(
        "forecast_date", "lead_time", "lat", "lon", "mean_std"
    )

    # Concatenate truth data
    y_train = xr.concat(
        [ws10_truth[0], ws10_truth[1], ws10_truth[2], ws10_truth[3]],
        dim="forecast_date",
    )

    # Extract shape of the data
    n_days, n_lead_times, lat, long, mean_var = ws_train.shape

    # Calculate elapsed time and print it
    half_time = time.time()
    time_difference_half = half_time - start_time
    hours = int(time_difference_half // 3600)
    minutes = int((time_difference_half % 3600) // 60)
    seconds = int(time_difference_half % 60)
    formatted_time_half = f" finished concatenation in:{hours} hours, {minutes} minutes, {seconds} seconds"
    print(f"{formatted_time_half}")

    # Create the datasets within the h5 file for 'train' and 'truth' data
    try:
        train = f.create_dataset(
            name_train,
            shape=(n_days, n_lead_times, lat, long, mean_var),
            dtype=np.float32,
            compression="gzip",
            compression_opts=9,
        )
    except:
        del f[name_train]  # if dataset already exists, delete it
        train = f.create_dataset(
            name_train,
            shape=(n_days, n_lead_times, lat, long, mean_var),
            dtype=np.float32,
            compression="gzip",
            compression_opts=9,
        )
    try:
        truth = f.create_dataset(
            name_truth,
            shape=(n_days, n_lead_times, lat, long),
            dtype=np.float32,
            compression="gzip",
            compression_opts=9,
        )
    except:
        del f[name_truth]  # if dataset already exists, delete it
        truth = f.create_dataset(
            name_truth,
            shape=(n_days, n_lead_times, lat, long),
            dtype=np.float32,
            compression="gzip",
            compression_opts=9,
        )

    # Populate the h5 file with the data
    for i in range(n_days):
        train[i, ...] = ws_train.loc[i, ...]
        truth[i, ...] = y_train[i, ...]

    # Close the h5 file
    f.close()

<IPython.core.display.Javascript object>

In [48]:
dat_raw[0].predictions.shape[0]

357

<IPython.core.display.Javascript object>

In [50]:
n_days_shape = []
for year in range(4):
    dat_raw = ldr.load_data_raw()[year]
    n_days_shape.append(dat_raw.predictions.shape[0])

<IPython.core.display.Javascript object>

In [51]:
n_days_shape

[357, 357, 358, 357]

<IPython.core.display.Javascript object>

In [ ]:
# My Methods
import data.raw.load_data_raw as ldr

def main():

    # Define path and file names for the h5 file to be created
    path = "/Data/Delong_BA_Data/Mean_ens_std/ws10_train.h5"
    f = h5py.File(path, "a")
    name_train = "ws10_train"
    name_truth = "ws10_truth"
    
    # Getting all n_days shapes of raw_data
    n_days_shape = []
    for year in (range(4)):
        dat_raw = ldr.load_data_raw()[year]
        n_days_shape.append(dat_raw.predictions.shape[0])
    
    # Load raw data for the years 2018-2021
    # process one year at a time
    for year in tqdm(range(4)):
        dat_raw = ldr.load_data_raw()[year]

        # Compute the magnitude (absolute value) of wind speed predictions and truths
        ws10_pred = np.hypot(dat_raw.predictions.isel(var=0), dat_raw.predictions.isel(var=1))
        ws10_tru = np.hypot(dat_raw.ground_truth.isel(var=0), dat_raw.ground_truth.isel(var=1))

        # Calculate mean and standard deviation of wind speed predictions
        ws10_pred_mean = ws10_pred.mean(dim="ens")
        ws10_pred_std = ws10_pred.std(dim="ens")

        # Concatenate mean and standard deviation data along new 'mean_std' dimension
        ws_train = xr.concat([ws10_pred_mean, ws10_pred_std], dim="mean_std")
        ws_train = ws_train.transpose("forecast_date", "lead_time", "lat", "lon", "mean_std")
        
        
        # Create the datasets within the h5 file for 'train' and 'truth' data
        # append data to existing dataset, or create it if it doesn't exist
        try:
            train = f[name_train]
        except:
            train_shape = (sum(n_days_shape), *ws_train.shape[1:])  # adjust dimensions as per your data
            train = f.create_dataset(name_train, shape=train_shape, dtype=np.float32, compression="gzip", compression_opts=9)
        try:
            truth = f[name_truth]
        except:
            truth_shape = (sum(n_days_shape), *ws10_tru.shape[1:])
            truth = f.create_dataset(name_truth, shape=truth_shape, dtype=np.float32, compression="gzip", compression_opts=9)

        # Populate the h5 file with the data
        train[sum(n_days_shape[0:year]):sum(n_days_shape[0:(year + 1)]), ...] = ws_train
        truth[sum(n_days_shape[0:year]):sum(n_days_shape[0:(year + 1)]), ...] = ws10_tru

    # Close the h5 file
    f.close()


In [49]:
test = [0, 1, 2, 3, 4, 5]
sum(test)

15

<IPython.core.display.Javascript object>

In [60]:
sum(n_days_shape[0:0])

0

<IPython.core.display.Javascript object>

In [61]:
sum(n_days_shape[0:1])

357

<IPython.core.display.Javascript object>